In [106]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import math

In [107]:
ltable = pd.read_csv('ltable.csv',encoding = "ISO-8859-1")
rtable = pd.read_csv('rtable.csv',encoding = "ISO-8859-1")
test = pd.read_csv('test.csv', encoding = "ISO-8859-1")
train = pd.read_csv('train.csv', encoding = "ISO-8859-1")
ltable.price = pd.to_numeric(ltable.price)
rtable.price = pd.to_numeric(rtable.price)

In [328]:
trainlabel = []
for row in train.iterrows():
    #need to figure out the threshhold for when the label = 1
    
    #compare titles
    trainltitle = ltable.title[row[1].ltable_id]
    trainrtitle = rtable.title[row[1].rtable_id]
    traintitleratio = fuzz.token_set_ratio(str(trainltitle), str(trainrtitle))
    traintitleratio = traintitleratio / 100
    
    if (traintitleratio > 0.95):
        traintitlelabel = 1
    else:
        traintitlelabel = 0
    
    #compare categories
    trainlcategory = ltable.category[row[1].ltable_id]
    trainrcategory = rtable.category[row[1].rtable_id]
    traincatratio = fuzz.token_set_ratio(str(trainlcategory), str(trainrcategory))
    traincatratio = traincatratio / 100
    
    if (traincatratio > 0.87):
        traincatlabel = 1
    else:
        traincatlabel = 0
    
    #compare brands
    trainlbrand = ltable.brand[row[1].ltable_id]
    trainrbrand = rtable.brand[row[1].rtable_id]
    trainbrandratio = fuzz.token_set_ratio(str(trainlbrand), str(trainrbrand))
    trainbrandratio = trainbrandratio / 100
    
    if (trainbrandratio > 0.87):
        trainbrandlabel = 1
    else:
        trainbrandlabel = 0
    
    #compare modelnumber
    trainlmodel = ltable.modelno[row[1].ltable_id]
    trainrmodel = rtable.modelno[row[1].rtable_id]
    trainmodelratio = fuzz.token_set_ratio(str(trainlmodel), str(trainrmodel))
    trainmodelratio = trainmodelratio / 100
    
    if (trainmodelratio > 0.95):
        trainmodellabel = 1
    elif ((str(trainlmodel) in trainrtitle) or (str(trainrmodel) in trainltitle)):
        trainmodellabel = 1
    else:
        trainmodellabel = 0
    
    #compare price
    trainlprice = ltable.price[row[1].ltable_id]
    trainrprice = rtable.price[row[1].rtable_id]
    
    if (math.isnan(trainlprice) or math.isnan(trainrprice)):
        pricelabel = 0
    else:
        priceratio = min(trainlprice, trainrprice) / max(trainlprice, trainrprice)
    
    if (trainpriceratio > 0.55):
        trainpricelabel = 1
    else:
        trainpricelabel = 0
    
    trainsumlabel = traintitlelabel + traincatlabel + trainbrandlabel + trainmodellabel + trainpricelabel
    
    if (trainsumlabel >= 4):
        trainlabel.append(1)
    else:
        trainlabel.append(0)


In [329]:
labelcolumn = train.loc[:, train.columns == 'label']

In [330]:
count = 0
for i in range(labelcolumn.label.size):
    if (labelcolumn.trainlabel[i] == trainlabel[i]):
        count += 1

In [331]:
count = count / trainlabel.label.size

In [332]:
print(count)

0.9327178626381312


In [337]:
label = []
for row in test.iterrows():
    #need to figure out the threshhold for when the label = 1
    
    #compare titles
    ltitle = ltable.title[row[1].ltable_id]
    rtitle = rtable.title[row[1].rtable_id]
    titleratio = fuzz.token_set_ratio(str(ltitle), str(rtitle))
    titleratio = titleratio / 100
    
    if (titleratio > 0.95):
        titlelabel = 1
    else:
        titlelabel = 0
    
    #compare categories
    lcategory = ltable.category[row[1].ltable_id]
    rcategory = rtable.category[row[1].rtable_id]
    catratio = fuzz.token_set_ratio(str(lcategory), str(rcategory))
    catratio = catratio / 100
    
    if (catratio > 0.87):
        catlabel = 1
    else:
        catlabel = 0
    
    #compare brands
    lbrand = ltable.brand[row[1].ltable_id]
    rbrand = rtable.brand[row[1].rtable_id]
    brandratio = fuzz.token_set_ratio(str(lbrand), str(rbrand))
    brandratio = brandratio / 100
    
    if (brandratio > 0.87):
        brandlabel = 1
    else:
        brandlabel = 0
    
    #compare modelnumber
    lmodel = ltable.modelno[row[1].ltable_id]
    rmodel = rtable.modelno[row[1].rtable_id]
    modelratio = fuzz.token_set_ratio(str(lmodel), str(rmodel))
    modelratio = modelratio / 100
    
    if (modelratio > 0.95):
        modellabel = 1
    elif ((str(lmodel) in rtitle) or (str(rmodel) in ltitle)):
        modellabel = 1
    else:
        modellabel = 0
    
    #compare price
    lprice = ltable.price[row[1].ltable_id]
    rprice = rtable.price[row[1].rtable_id]
    
    if (math.isnan(lprice) or math.isnan(rprice)):
        pricelabel = 0
    else:
        priceratio = min(lprice, rprice) / max(lprice, rprice)
    
    if (priceratio > 0.55):
        pricelabel = 1
    else:
        pricelabel = 0
    
    sumlabel = titlelabel + catlabel + brandlabel + modellabel + pricelabel
    
    if (sumlabel >= 4):
        label.append(1)
    else:
        label.append(0)


In [338]:
testid = test['id']

In [339]:
submission = pd.DataFrame({
    "label" : label,
    "id" : testid
})
submission.to_csv("Submission.csv", index = False, encoding = 'utf-8')